# Week 7: Joining Data Sources - Pandas Merging & Joining

**PORA Academy Cohort 5 - Data Analytics & AI Bootcamp**

**Wednesday Python Session - September 24, 2025**

---

## Learning Objectives
By the end of this session, you will be able to:
1. Understand why data is split across multiple files (normalization)
2. Use `pd.merge()` to combine DataFrames with different join types
3. Apply inner, left, right, and outer merges appropriately
4. Handle duplicate keys and missing matches in merged data
5. Chain multiple merge operations for complex analysis
6. Understand the relationship between pandas merge and SQL JOINs

## Excel Bridge Concept
**Excel Equivalent**: VLOOKUP, INDEX-MATCH, and Power Query merge operations

In Excel, you use VLOOKUP to bring data from one table into another. In pandas, we use `merge()` and `join()` functions to combine DataFrames. The concepts are identical to SQL JOINs you'll learn tomorrow!

## Part 1: Loading and Understanding Relational Data

### Why Data is Split Across Files

**Concept**: Instead of one massive Excel file with duplicate information, we split related data into multiple files.

**Benefits**:
- **No Redundancy**: Customer address appears once, not in every order row
- **Easy Updates**: Change customer info in one place
- **Better Performance**: Smaller files load faster

**Real-World Analogy**: Your phone contacts app doesn't save complete contact info in every message. Messages reference contacts by ID, keeping data separate but connected.

In [ ]:
import pandas as pd
import numpy as np

# Load all datasets
customers = pd.read_csv('../datasets/customers.csv')
orders = pd.read_csv('../datasets/orders.csv')
order_items = pd.read_csv('../datasets/order_items.csv')
products = pd.read_csv('../datasets/products.csv')
sellers = pd.read_csv('../datasets/sellers.csv')
reviews = pd.read_csv('../datasets/order_reviews.csv')
payments = pd.read_csv('../datasets/order_payments.csv')

# Convert date columns
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])

print("Datasets loaded successfully!")
print(f"\nCustomers shape: {customers.shape}")
print(f"Orders shape: {orders.shape}")
print(f"Order items shape: {order_items.shape}")

In [ ]:
# Explore the structure
print("Customers columns:", customers.columns.tolist())
print("\nOrders columns:", orders.columns.tolist())
print("\nFirst few customers:")
customers.head()

In [ ]:
print("First few orders:")
orders.head()

### Understanding Keys
- **Primary Key**: Unique identifier for each row (like `customer_id`)
- **Foreign Key**: Column that references a primary key in another DataFrame
- **Join Column**: The column used to connect DataFrames

## Part 2: Inner Merge - The Foundation

**Inner Merge**: Returns only rows where keys exist in BOTH DataFrames.

**Excel Analogy**: Like VLOOKUP, but only keeps rows where a match is found. No match? Row is excluded.

**Visual Representation**:
```
DataFrame A    DataFrame B     Inner Merge
-----------    -----------     -----------
  ┌─────┐        ┌─────┐         ┌─────┐
  │ A   │   +    │ B   │    =    │ A∩B │  (overlap only)
  └─────┘        └─────┘         └─────┘
```

In [ ]:
# Basic Inner Merge: Orders with Customer Information
# Business Question: "Show me orders with customer city and state information"

orders_customers = pd.merge(
    orders,
    customers,
    on='customer_id',
    how='inner'
)

# Select relevant columns
result = orders_customers[[
    'order_id',
    'order_purchase_timestamp',
    'order_status',
    'customer_city',
    'customer_state'
]]

print(f"Orders before merge: {len(orders)}")
print(f"Customers before merge: {len(customers)}")
print(f"Rows after inner merge: {len(orders_customers)}")
print("\nMerged data:")
result.head()

### Three-DataFrame Merge

**Business Question**: "What products were purchased in each order?"

In [ ]:
# Step 1: Merge orders with order_items
orders_items = pd.merge(
    orders,
    order_items,
    on='order_id',
    how='inner'
)

# Step 2: Merge result with products
orders_items_products = pd.merge(
    orders_items,
    products,
    on='product_id',
    how='inner'
)

# Filter and display
delivered_orders = orders_items_products[
    orders_items_products['order_status'] == 'delivered'
]

result = delivered_orders[[
    'order_id',
    'order_purchase_timestamp',
    'product_category_name',
    'price',
    'freight_value'
]]

print(f"Delivered orders with product details:")
result.head(10)

### Method Chaining Approach

More concise way to chain multiple merges:

In [ ]:
# Same result using method chaining
result = (orders
    .merge(order_items, on='order_id', how='inner')
    .merge(products, on='product_id', how='inner')
    .query('order_status == "delivered"')
    [['order_id', 'order_purchase_timestamp', 'product_category_name', 'price', 'freight_value']]
)

print("Method chaining result:")
result.head(10)

## Part 3: Left Merge - Keeping All Records

**Left Merge**: Returns ALL rows from the left DataFrame and matching rows from the right. If no match, right columns are NaN.

**Excel Analogy**: Like VLOOKUP with IFERROR - you keep all original rows, showing blanks (NaN) when no match found.

**Visual Representation**:
```
DataFrame A    DataFrame B     Left Merge
-----------    -----------     ----------
  ┌─────┐        ┌─────┐       ┌─────┐
  │  A  │   +    │ B   │   =   │ All │  (all of A)
  └─────┘        └─────┘       │  A  │
                               └─────┘
```

### When to Use Left Merge
- All records from the main DataFrame regardless of matches
- To identify records WITHOUT matches (using `.isna()`)
- To preserve your primary dataset while adding supplementary info

In [ ]:
# Left Merge: All Orders with Optional Reviews
# Business Question: "Show all orders, including those without reviews"

orders_reviews = pd.merge(
    orders,
    reviews,
    on='order_id',
    how='left'
)

# Filter for delivered orders
delivered = orders_reviews[orders_reviews['order_status'] == 'delivered']

# Add review status column
delivered['review_status'] = delivered['review_id'].apply(
    lambda x: 'Has Review' if pd.notna(x) else 'No Review'
)

result = delivered[[
    'order_id',
    'order_status',
    'order_purchase_timestamp',
    'review_score',
    'review_creation_date',
    'review_status'
]]

print("Orders with review status:")
print(result.head(10))
print("\nReview completion breakdown:")
print(delivered['review_status'].value_counts())

### Finding Missing Data with Left Merge

In [ ]:
# Business Question: "Which delivered orders have no customer reviews?"

orders_reviews = pd.merge(
    orders,
    reviews,
    on='order_id',
    how='left'
)

# Filter for delivered orders without reviews
missing_reviews = orders_reviews[
    (orders_reviews['order_status'] == 'delivered') &
    (orders_reviews['review_id'].isna())
]

result = missing_reviews[[
    'order_id',
    'customer_id',
    'order_purchase_timestamp',
    'order_delivered_customer_date'
]]

print(f"Orders without reviews: {len(missing_reviews)}")
print("\nSample of orders missing reviews:")
result.head()

### Data Completeness Analysis

In [ ]:
# Business Question: "What percentage of delivered orders have reviews?"

orders_reviews = pd.merge(
    orders.query('order_status == "delivered"'),
    reviews,
    on='order_id',
    how='left'
)

total_orders = len(orders_reviews)
reviewed_orders = orders_reviews['review_id'].notna().sum()
completion_rate = (reviewed_orders / total_orders) * 100

print(f"Total delivered orders: {total_orders}")
print(f"Orders with reviews: {reviewed_orders}")
print(f"Review completion rate: {completion_rate:.2f}%")

## Part 4: Right and Outer Merges

### Right Merge
**Right Merge**: Opposite of left merge - keeps all rows from RIGHT DataFrame.

**Note**: Less common in practice. You can reverse DataFrames and use left merge instead.

In [ ]:
# Right merge example
reviews_orders = pd.merge(
    orders,
    reviews,
    on='order_id',
    how='right'
)

print(f"Right merge result: {len(reviews_orders)} rows")
print("\nSame as reversing and using left merge:")

reviews_orders_alt = pd.merge(
    reviews,
    orders,
    on='order_id',
    how='left'
)

print(f"Alternative approach: {len(reviews_orders_alt)} rows")

### Outer Merge (Full Outer Join)
**Outer Merge**: Returns ALL rows from BOTH DataFrames, with NaN where no match exists.

**Use Cases**: Data reconciliation, finding all mismatches

In [ ]:
# Full outer merge: Find all order/review mismatches
all_data = pd.merge(
    orders,
    reviews,
    on='order_id',
    how='outer',
    indicator=True  # Shows merge source
)

# Find mismatches
mismatches = all_data[all_data['_merge'] != 'both']

print("Merge status breakdown:")
print(all_data['_merge'].value_counts())
print("\nMismatches:")
print(mismatches[['order_id', 'order_status', 'review_id', '_merge']].head())

## Part 5: Advanced Merge Scenarios

### Multi-DataFrame Analysis

In [ ]:
# Business Question: "Analyze SP (São Paulo) orders: products, payments, and reviews"

sp_analysis = (orders
    .merge(customers, on='customer_id', how='inner')
    .merge(order_items, on='order_id', how='inner')
    .merge(products, on='product_id', how='inner')
    .merge(payments, on='order_id', how='left')
    .merge(reviews, on='order_id', how='left')
    .query('customer_state == "SP" and order_status == "delivered"')
)

result = sp_analysis[[
    'customer_city',
    'order_id',
    'product_category_name',
    'price',
    'payment_type',
    'review_score'
]]

print("São Paulo orders analysis:")
result.head(10)

### Aggregations with Merged Data

In [ ]:
# Business Question: "Average review score by product category"

category_reviews = (products
    .merge(order_items, on='product_id', how='inner')
    .merge(orders, on='order_id', how='inner')
    .merge(reviews, on='order_id', how='left')
    .query('order_status == "delivered" and product_category_name.notna()')
)

# Group and calculate
summary = category_reviews.groupby('product_category_name').agg({
    'order_id': 'nunique',
    'order_item_id': 'count',
    'review_score': 'mean',
    'price': 'mean'
}).rename(columns={
    'order_id': 'total_orders',
    'order_item_id': 'total_items',
    'review_score': 'avg_review_score',
    'price': 'avg_price'
}).round(2)

# Filter and sort
top_categories = summary[summary['total_orders'] >= 1].sort_values('avg_review_score', ascending=False)

print("Category performance:")
top_categories.head(10)

### Handling Duplicate Keys (One-to-Many Relationships)

In [ ]:
# Business Question: "Orders with multiple items - understanding one-to-many relationships"

orders_with_items = pd.merge(
    orders,
    order_items,
    on='order_id',
    how='inner'
)

print(f"Original orders: {len(orders)}")
print(f"After merge: {len(orders_with_items)}")
print(f"\nDuplicates created by one-to-many relationship")

# Count items per order
items_per_order = orders_with_items.groupby('order_id').size().reset_index(name='item_count')
print("\nOrders with multiple items:")
print(items_per_order[items_per_order['item_count'] > 1].head())

## Part 6: Common Merge Issues and Solutions

### Issue 1: Key Column Name Mismatch

In [ ]:
# Example: Different column names for same key
# If orders had 'customer_id' but another df had 'cust_id'

# Solution: Use left_on and right_on
# result = pd.merge(
#     orders,
#     other_df,
#     left_on='customer_id',
#     right_on='cust_id',
#     how='inner'
# )
# result = result.drop('cust_id', axis=1)  # Drop duplicate column

print("Use left_on and right_on when key column names differ")

### Issue 2: Unexpected Number of Rows

In [ ]:
# Before merge: Check for duplicates in key columns
print("Duplicate order_ids in orders:", orders['order_id'].duplicated().sum())
print("Duplicate order_ids in reviews:", reviews['order_id'].duplicated().sum())

# After merge: Validate
merged = pd.merge(orders, order_items, on='order_id', how='inner')
print(f"\nOrders before merge: {len(orders)}")
print(f"Rows after merge: {len(merged)}")
print(f"Ratio: {len(merged) / len(orders):.2f}x")

### Issue 3: NaN Values After Merge

In [ ]:
# Check for NaN values
merged = pd.merge(orders, reviews, on='order_id', how='left')

print("Missing review scores:")
print(merged['review_score'].isna().sum())

# Fill NaN with appropriate values
merged['review_score_filled'] = merged['review_score'].fillna(0)
merged['review_status'] = merged['review_score'].apply(
    lambda x: 'Reviewed' if pd.notna(x) else 'Not Reviewed'
)

print("\nReview status distribution:")
print(merged['review_status'].value_counts())

## Comparison: Pandas Merge vs SQL JOIN

| pandas | SQL | Purpose |
|--------|-----|---------|  
| `how='inner'` | INNER JOIN | Only matches |
| `how='left'` | LEFT JOIN | All from left |
| `how='right'` | RIGHT JOIN | All from right |
| `how='outer'` | FULL OUTER JOIN | All from both |
| `on='column'` | ON table1.column = table2.column | Join condition |
| `indicator=True` | (manual check) | Track merge source |

**Tomorrow's SQL class will teach the same concepts with SQL syntax!**

## Key Takeaways

### Merge Types Quick Reference
- **Inner**: Only matching rows from both DataFrames
- **Left**: All from left + matching from right (NaN for no match)
- **Right**: All from right + matching from left
- **Outer**: All rows from both DataFrames

### Best Practices
1. **Check key columns before merging** (duplicates, NaN values)
2. **Validate merge results** (row counts, NaN patterns)
3. **Use left merge** when you need to preserve all primary records
4. **Chain merges logically** (start with main DataFrame)
5. **Use `indicator=True`** for debugging merge issues
6. **Handle NaN values appropriately** after merging

### Real-World Applications
- **E-commerce**: Combining orders, products, and customers
- **Marketing**: Joining campaign data with customer behavior
- **Finance**: Merging transactions with account information
- **Healthcare**: Connecting patient records across systems

---

**Next Session**: Thursday SQL class will cover JOINs - the same concepts you learned today but with SQL syntax and the same Olist database!